In [2]:
#import csv for files and numpy for math and arrays
import csv as csv
import numpy as np

#open csv file in to Python object
csv_file_object = csv.reader(open('train.csv', 'rb'))
header = csv_file_object.next()

data = []
for row in csv_file_object:
    data.append(row)
data = np.array(data)

In [3]:
number_passengers = np.size(data[0::,1].astype(np.float))
number_survived = np.sum(data[0::,1].astype(np.float))
proportion_survivors = number_survived / number_passengers

In [4]:
women_only_stats = data[0::,4] == 'female'
men_only_stats = data[0::,4] != 'female'

In [5]:
women_onboard = data[women_only_stats,1].astype(np.float)
men_onboard = data[men_only_stats,1].astype(np.float)

proportion_women_survived = np.sum(women_onboard) / np.size(women_onboard)
proportion_men_survived = np.sum(men_onboard) / np.size(men_onboard)

In [6]:
print 'proportion of women who survived is %s' % proportion_women_survived
print 'proportion of men who survived is %s' % proportion_men_survived

proportion of women who survived is 0.742038216561
proportion of men who survived is 0.188908145581


In [7]:
#reading test data and writing gender model to csv
test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

In [8]:
prediction_file = open('genderbasedmodel.csv', 'wb')
prediction_file_object = csv.writer(prediction_file)

In [9]:
prediction_file_object.writerow(['PassengerId', 'Survived'])
for row in test_file_object:
    if row[3] == 'female':
        prediction_file_object.writerow([row[0], '1'])
    else:
        prediction_file_object.writerow([row[0], '0'])
test_file.close()
prediction_file.close()

In [10]:
#binning ticket prices
fare_ceiling = 40
data[data[0::,9].astype(np.float) >= fare_ceiling, 9] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

number_of_classes = len(np.unique(data[0::, 2]))

survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

In [26]:
#loop through each class
for i in xrange(number_of_classes):
    for j in xrange(number_of_price_brackets):
        
        women_only_stats = data[(data[0::, 4] == 'female') \
                                & (data[0::, 2].astype(np.float) == i + 1) \
                               & (data[0:, 9].astype(np.float) >= j * fare_bracket_size) \
                               & (data[0:, 9].astype(np.float) < (j+1) * fare_bracket_size) \
                               , 1]
        
        men_only_stats = data[(data[0::, 4] != 'female') \
                                & (data[0::, 2].astype(np.float) == i + 1) \
                               & (data[0:, 9].astype(np.float) >= j * fare_bracket_size) \
                               & (data[0:, 9].astype(np.float) < (j+1) * fare_bracket_size) \
                               , 1]
        
        survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float)) 
        survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))

survival_table[survival_table != survival_table] = 0

In [27]:
print survival_table

[[[ 0.          0.          0.83333333  0.97727273]
  [ 0.          0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.          0.          0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]


In [28]:
print women_only_stats

['1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']


In [29]:
survival_table[survival_table < 0.5] = 0
survival_table[survival_table >= 0.5] = 1

In [30]:
print survival_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [41]:
#create csv for test.csv using genderclassmodel
test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()
predictions_file = open('genderclassmodel.csv', 'wb')
p = csv.writer(predictions_file)
p.writerow(['PassengerId', 'Survived'])

In [42]:
for row in test_file_object:
    for j in xrange(number_of_price_brackets):
        try:
            row[8] = float(row[8])
        except:
            bin_fare = 3 - float(row[1])
            break
        if row[8] > fare_ceiling:
            bin_fare = number_of_price_brackets - 1
            break
        if row[8] <= j * fare_bracket_size and row[8] < (j + 1) * fare_bracket_size:
            bin_fare = j
            break
    if row[3] == 'female':
        p.writerow([row[0], '%d' % int(survival_table[0, float(row[1]) - 1, bin_fare])])
    else:
        p.writerow([row[0], '%d' % int(survival_table[1, float(row[1]) - 1, bin_fare])])


F:\Data Science\Anaconda 2.7\lib\site-packages\ipykernel\__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
F:\Data Science\Anaconda 2.7\lib\site-packages\ipykernel\__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [43]:
# Close out the files
test_file.close()
predictions_file.close()

In [44]:
print survival_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]
